# Bernstein-Vazirani algorithm (Q# code)

This notebook implements quantum Bernstein-Vazirani algorithm in Q#.

See [here](https://tsmatz.wordpress.com/2019/02/21/quantum-computing-programming-qsharp-for-phase-kickback/) for mathematical background.

*back to [index](https://github.com/tsmatz/quantum-algorithms-qsharp)*

In [1]:
import qsharp

Preparing Q# environment...
.

In [2]:
# declare function
BernsteinVaziraniTestCase: any = None

In [3]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation BernsteinVaziraniTestCase (n : Int, patternInt : Int) : Int {
    let pattern = IntAsBoolArray(patternInt, n);

    use qubits = Qubit[n + 1] { // all n+1 qubits are initialized as |0〉

        // set last qubits[n] as |1〉
        X(qubits[n]);

        // set qubits[i] = |+〉 (i=0 ... n-1) and qubits[n] = |-〉
        ApplyToEach(H, qubits);

        // apply unitary transformation for qubits
        let Uf = ParityOperation(pattern);
        Uf(qubits);

        // apply Hadamard again
        ApplyToEach(H, qubits[0 .. n - 1]);

        // measure and reset qubit
        let resultArray = ForEach(MResetZ, qubits[0 .. n - 1]);
        let resultBool = ResultArrayAsBoolArray(resultArray);
        let resultInt = BoolArrayAsInt(resultBool);

        // reset last qubit
        Reset(qubits[n]);

        return resultInt;
    }
}

operation ParityOperationImpl (pattern : Bool[], qs : Qubit[]) : Unit {    
    let n = Length(pattern);
    for idx in 0 .. n - 1 {
        if (pattern[idx]) {
            Controlled X([qs[idx]], qs[n]);
        }
    }
}
  
function ParityOperation (pattern : Bool[]) : (Qubit[] => Unit) {
    return ParityOperationImpl(pattern, _);
}

In [5]:
import random

n = 4
parity = random.randint(0, 1 << n - 1)
measuredParity = BernsteinVaziraniTestCase.simulate(n=n, patternInt=parity)
print("Actual Parity is {}".format(parity))
print("Measured Parity is {}".format(measuredParity))

Actual Parity is 11
Measured Parity is 11
